In [1]:
import pandas as pd
import os
import seaborn as sns

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from datetime import datetime

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias
import funcion_relacion_terrenos_w_ui
import funcion_dataframe_terrenos

<h3>Ejecución de Funciones</h3>

Funciones que relacionan:

<ol>
<li> Terrenos Gestor con Terrenos SIG = Terrenos w GIS
<li> Terrenos w GIS con Terrenos Reconocimiento (PENDIENTE)
</ol>

In [ ]:

arcpy.env.overwriteOutput = True

# TODO: LLamado a Terrenos
df_terrenos_unificados, ruta_terrenos_unificados = funcion_dataframe_terrenos.dataframe_terrenos()
print("Se ejecuta la función")

# TODO: LLamado a UI
arcpy.env.workspace = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\O_Apoyo_Construccion_UIntervencion\UI_Finales\UI_Finales.gdb"
nDataset = 'UI_Unificadas'
fcUI = 'UI_Unificadas'

for dataset in arcpy.ListDatasets(wild_card=nDataset):
    for capa in arcpy.ListFeatureClasses(feature_dataset=dataset, wild_card=fcUI):
        ruta_fcUI = os.path.join(arcpy.env.workspace,dataset,capa)

# TODO: Almacenamiento de capas temporales
ruta_bdScrach = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_0_Identificacion_Predial_Total_Ha_Actualizadas\Base_Datos\Terrenos_w_UI.gdb"

funcion_relacion_terrenos_w_ui.funcion_terreno_w_ui(ruta_terrenos_unificados, ruta_bdScrach, ruta_fcUI)

<h3>Variables que alimentan la construcción de la capa Terrenos Final con la Capa de Hitos (y su relación creada anteriormente con las Geo UI)</h3>

In [3]:
RUTA_TERRENOS_W_GIS_BCGS = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada\TERRENO_UNIFICADO_GIS_BCGS"
RUTA_RELACION_NPN_HITO  = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\relacion_terreno_ui"

RUTA_BDCONSOLIDADA = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada"
NOMBRE_CAPA_TERRENO_X_HITO = 'TERRENO_POR_HITO'

dict_municipios = {'13442':'MariaLaBaja',
    '08078':'Baranoa',
    '08606':'Repelon',
    '08634':'Sabanagrande'}

In [ ]:
df_terrenosWgis = pd.DataFrame.spatial.from_featureclass(RUTA_TERRENOS_W_GIS_BCGS)
df_npnXhito = pd.DataFrame.spatial.from_table(RUTA_RELACION_NPN_HITO)
df_ui = pd.DataFrame.spatial.from_featureclass(ruta_fcUI)

# TODO: Aqui se relaciona la capa de Terrenos Final con la Relación NPN X Hito creada en fases anteriores del código
df_terrenosgisXhito = pd.merge(left=df_terrenosWgis,
    right=df_npnXhito,
    on='codigo',
    how='inner')

df_terrenosgisXhito['nombre_municipio'] = None

for llave, valor in dict_municipios.items():
    for columna in df_terrenosgisXhito.columns:
        if columna == 'codigo':
            df_terrenosgisXhito.loc[df_terrenosgisXhito[columna].str[:5] == llave, 'nombre_municipio'] = valor

estandarizacion_columnas = ['nombre_municipio','terreno_codigo','etiqueta','espacio_de_nombres','local_id','bcgs_creador','bcgs_ultima_edicion','bcgs_fecha_ultima_edicion','codigo','codigo_anterior','sig_creador','sig_ultima_edicion','sig_fecha_ultima_edicion','fuente','area_ha_cmt12','id_ui','SHAPE']
df_terrenosgisXhito = df_terrenosgisXhito[estandarizacion_columnas]

# TODO: Estandarización del datafre de las UI
estandarizacion_ui = ['ID_UI','Nombre_UI','Meta_Hito']
df_ui = df_ui[estandarizacion_ui]

df_terrenoXhitoXmeta = pd.merge(left=df_terrenosgisXhito,
    right=df_ui,
    left_on='id_ui',
    right_on='ID_UI',
    how='inner')

# TODO: Se estandariza la capa de Terrenos Final
estandarizacion_columnas_meta = ['nombre_municipio','terreno_codigo','etiqueta','espacio_de_nombres','local_id','bcgs_creador','bcgs_ultima_edicion','bcgs_fecha_ultima_edicion','codigo','codigo_anterior','sig_creador','sig_ultima_edicion','sig_fecha_ultima_edicion','fuente','area_ha_cmt12','id_ui','Nombre_UI','Meta_Hito','SHAPE']
df_terrenoXhitoXmeta = df_terrenoXhitoXmeta[estandarizacion_columnas_meta]

# TODO: La capa definitiva se almacena en la BD Consolidada Local
df_terrenoXhitoXmeta.spatial.to_featureclass(location=os.path.join(RUTA_BDCONSOLIDADA, NOMBRE_CAPA_TERRENO_X_HITO))
print(f"Se genera la capa {NOMBRE_CAPA_TERRENO_X_HITO}")